In [ ]:
!pip install -U "ray[data,train,tune,serve]"
!pip install tqdm
!pip install pyarrow

In [ ]:
import os
import boto3
import botocore

aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_S3_ENDPOINT')
region_name = os.environ.get('AWS_DEFAULT_REGION')
bucket_name = os.environ.get('AWS_S3_BUCKET')

if not all([aws_access_key_id, aws_secret_access_key, endpoint_url, region_name, bucket_name]):
    raise ValueError("One or data connection variables are empty.  "
                     "Please check your data connection to an S3 bucket.")

session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key)

s3_resource = session.resource(
    's3',
    config=botocore.client.Config(signature_version='s3v4'),
    endpoint_url=endpoint_url,
    region_name=region_name)

bucket = s3_resource.Bucket(bucket_name)

bucket.upload_file("card_transdata.csv", "data/card_transdata.csv")

In [ ]:
import os
from pyarrow import fs
import ray

fs = fs.S3FileSystem(access_key=aws_access_key_id,
                     secret_key=aws_secret_access_key,
                     region=region_name,
                     endpoint_override=endpoint_url)

ds = ray.data.read_csv(filesystem=fs,
                       paths=f"s3://{os.environ.get('AWS_S3_BUCKET')}/data/card_transdata.csv")
ds.schema()